In [304]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import numpy as np
import pprint

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Scraping For Yearly Team Slugs

In [159]:
urls = ['https://www.basketball-reference.com/leagues/NBA_2018.html',
    'https://www.basketball-reference.com/leagues/NBA_2017.html',
       'https://www.basketball-reference.com/leagues/NBA_2016.html']

In [160]:
all_slugs = []

In [161]:
for i in urls:
    url = i
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find('table', {'id':'confs_standings_E'}).find('tbody')
    table2 = soup.find('table', {'id':'confs_standings_W'}).find('tbody')
    for row in table.find_all('tr'):
        all_slugs.append(row.find('th', {'data-stat':'team_name'}).find('a').attrs['href'])
    for row in table2.find_all('tr'):
        all_slugs.append(row.find('th', {'data-stat':'team_name'}).find('a').attrs['href'])

In [162]:
urls2 = ['https://www.basketball-reference.com/leagues/NBA_2015.html',
       'https://www.basketball-reference.com/leagues/NBA_2014.html',
       'https://www.basketball-reference.com/leagues/NBA_2013.html',
       'https://www.basketball-reference.com/leagues/NBA_2012.html',
        'https://www.basketball-reference.com/leagues/NBA_2011.html',
        'https://www.basketball-reference.com/leagues/NBA_2010.html',
        'https://www.basketball-reference.com/leagues/NBA_2009.html',
        'https://www.basketball-reference.com/leagues/NBA_2008.html',
        'https://www.basketball-reference.com/leagues/NBA_2007.html']

In [163]:
for i in urls2:
    url = i
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find('table', {'id':'divs_standings_E'}).find('tbody')
    table2 = soup.find('table', {'id':'divs_standings_W'}).find('tbody')
    for row in table.find_all('tr', {'class':'full_table'}):
        all_slugs.append(row.find('th', {'data-stat':'team_name'}).find('a').attrs['href'])
    for row in table2.find_all('tr', {'class':'full_table'}):
        all_slugs.append(row.find('th', {'data-stat':'team_name'}).find('a').attrs['href'])

# Scraping Team Yearly Payrolls

In [166]:
team_yearly_payrolls = []

for i in all_slugs:
    url = 'https://www.basketball-reference.com' + i
    res = requests.get(url).content
    res = res.decode("utf-8")
    soup = BeautifulSoup(re.sub("<!--|-->","", res), 'lxml')
    season = soup.find('div', {'id':'meta'}).find('h1', {'itemprop':'name'}).find('span').text
    print(season)
    table = soup.find('table', {'id':'per_game'}).find('tbody')
    team_payroll = []
    team = i[7:10]
    for i in soup.find('table', {'id':'salaries2'}).find_all('tr'):
        if i.find('td', {'data-stat':'salary'}) == None:
            continue
        elif i.find('td', {'data-stat':'salary'}).text == '':
            continue
        else:
            team_payroll.append(i.find('td', {'data-stat':'salary'}).text)
    for i in range(0,len(team_payroll)):
        try:
            if team_payroll[i] == None:
                continue
            else:
                team_payroll[i] = team_payroll[i].replace(',','')
                team_payroll[i] = team_payroll[i].replace('$','')
        except TypeError:
            continue
    team_payroll = [sum([float(i) for i in team_payroll])]
    team_payroll.append(team)
    team_payroll.append(season)
    team_yearly_payrolls.append(team_payroll)
    

2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2013-14
2013-14
2013-14
2013-14
2013-14


In [339]:
team_yearly_payrolls[220]

[66205855.0, 'MIA', '2010-11']

# Scraping Player Data

In [373]:
stats_for_df = []
names = []

In [374]:
for i in all_slugs:
    url = 'https://www.basketball-reference.com' + i
    res = requests.get(url).content
    res = res.decode("utf-8")
    soup = BeautifulSoup(re.sub("<!--|-->","", res), 'lxml')
    season = soup.find('div', {'id':'meta'}).find('h1', {'itemprop':'name'}).find('span').text
    print(season)
    table = soup.find('table', {'id':'per_game'}).find('tbody')
    for player in table.find_all('tr'):
        stat = {}
        name = player.find('td', {'data-stat':'player'}).text
        slug = player.find('td', {'data-stat':'player'}).find('a').attrs['href']
        if name in names:
            continue
        else:
            player_url = 'https://www.basketball-reference.com' + slug
            player_res = requests.get(player_url).content
            player_res = player_res.decode("utf-8")
            player_soup = BeautifulSoup(re.sub("<!--|-->","", player_res), 'lxml')
            years = ['placeholder']
            try:
                if player_soup.find('table', {'id':'per_game'}).find('tbody') == None:
                    continue
                else:
                    table2 = player_soup.find('table', {'id':'per_game'}).find('tbody')
                if player_soup.find('table', {'id':'advanced'}).find('tbody') == None:
                    continue
                else:
                    table3 = player_soup.find('div', {'id':'content'}).find('table', {'id':'advanced'})
                if player_soup.find('div', {'id':'content'}).find('div', {'id':'div_all_salaries'}) == None:
                    continue
                else:
                    table4 = player_soup.find('table', {'id':'all_salaries'}).find('tbody')
                stat = {}
                if player_soup.find('table', {'id':'per_game'}).find('tbody') == None:
                    continue
                else:
                    for player in table2.find_all('tr'):
                        stat['Name'] = player_soup.find('div', {'id':'meta'}).find('h1', {'itemprop':'name'}).text
                        names.append(stat['Name'])
                        if player.find('th', {'data-stat':'season'}) == None:
                            continue
                        else:
                            season = player.find('th', {'data-stat':'season'}).find('a').text   
                            if season in years:
                                continue
                            stat['League'] = player.find('td', {'data-stat':'lg_id'}).find('a').text
                            stat['Position'] = player.find('td', {'data-stat':'pos'}).text
                            stat[season + ' Age'] = player.find('td', {'data-stat':'age'}).text
                            stat[season + ' Games Played'] = player.find('td', {'data-stat':'g'}).text
                            stat[season + ' Games Started'] = player.find('td', {'data-stat':'gs'}).text
                            stat[season + ' Minutes Per Game'] = player.find('td', {'data-stat':'mp_per_g'}).text
                            stat[season + ' Field Goals Per Game'] = player.find('td', {'data-stat':'fg_per_g'}).text
                            stat[season + ' Field Goal Attempts Per Game'] = player.find('td', {'data-stat':'fga_per_g'}).text
                            stat[season + ' Field Goal Percentage'] = player.find('td', {'data-stat':'fg_pct'}).text
                            stat[season + ' 3 Point Field Goals Per Game'] = player.find('td', {'data-stat':'fg3_per_g'}).text
                            stat[season + ' 3 Point Field Goal Attempts Per Game'] = player.find('td', {'data-stat':'fg3a_per_g'}).text
                            stat[season + ' 3 Point Field Goal Percentage'] = player.find('td', {'data-stat':'fg3_pct'}).text
                            stat[season + ' 2 Point Field Goals Per Game'] = player.find('td', {'data-stat':'fg2_per_g'}).text
                            stat[season + ' 2 Point Field Goal Attempts Per Game'] = player.find('td', {'data-stat':'fg2a_per_g'}).text
                            stat[season + ' 2 Point Field Goal Percentage'] = player.find('td', {'data-stat':'fg2_pct'}).text
                            stat[season + ' Effective Field Goal Percentage'] = player.find('td', {'data-stat':'efg_pct'}).text
                            stat[season + ' Free Throw Makes Per Game'] = player.find('td', {'data-stat':'ft_per_g'}).text
                            stat[season + ' Free Throw Attempts Per Game'] = player.find('td', {'data-stat':'fta_per_g'}).text
                            stat[season + ' Free Throw Percentage'] = player.find('td', {'data-stat':'ft_pct'}).text
                            stat[season + ' Offensive Rebounds Per Game'] = player.find('td', {'data-stat':'orb_per_g'}).text
                            stat[season + ' Defensive Rebounds Per Game'] = player.find('td', {'data-stat':'drb_per_g'}).text
                            stat[season + ' Total Rebounds Per Game'] = player.find('td', {'data-stat':'trb_per_g'}).text
                            stat[season + ' Assists Per Game'] = player.find('td', {'data-stat':'ast_per_g'}).text
                            stat[season + ' Steals Per Game'] = player.find('td', {'data-stat':'stl_per_g'}).text
                            stat[season + ' Blocks Per Game'] = player.find('td', {'data-stat':'blk_per_g'}).text
                            stat[season + ' Turnovers Per Game'] = player.find('td', {'data-stat':'tov_per_g'}).text
                            stat[season + ' Personal Fouls Per Game'] = player.find('td', {'data-stat':'pf_per_g'}).text
                            stat[season + ' Points Per Game'] = player.find('td', {'data-stat':'pts_per_g'}).text
                            stat[season + ' Years In League'] = len(years)
                            
                            if season in years:
                                continue
                            else:
                                years.append(season)

                if player_soup.find('table', {'id':'advanced'}) == None:
                    continue
                else:
                    for player in table3.find('tbody').find_all('tr'):
                        if player.find('th', {'data-stat':'season'}) == None:
                            continue
                        else:
                            season = player.find('th', {'data-stat':'season'}).find('a').text
                            stat[season + ' Total Minutes Played'] = player.find('td', {'data-stat':'mp'}).text
                            stat[season + ' Player Efficiency Rating'] = player.find('td', {'data-stat':'per'}).text
                            stat[season + ' True Shooting Percentage'] = player.find('td', {'data-stat':'ts_pct'}).text
                            stat[season + ' 3 Point Attempt Rate'] = player.find('td', {'data-stat':'fg3a_per_fga_pct'}).text
                            stat[season + ' Free Throw Attempt Rate'] = player.find('td', {'data-stat':'fta_per_fga_pct'}).text
                            stat[season + ' Offensive Rebound Percentage'] = player.find('td', {'data-stat':'orb_pct'}).text
                            stat[season + ' Defensive Rebound Percentage'] = player.find('td', {'data-stat':'drb_pct'}).text
                            stat[season + ' Total Rebound Percentage'] = player.find('td', {'data-stat':'trb_pct'}).text
                            stat[season + ' Assist Percentage'] = player.find('td', {'data-stat':'ast_pct'}).text
                            stat[season + ' Steal Percentage'] = player.find('td', {'data-stat':'stl_pct'}).text
                            stat[season + ' Block Percentage'] = player.find('td', {'data-stat':'blk_pct'}).text
                            stat[season + ' Turnover Percentage'] = player.find('td', {'data-stat':'tov_pct'}).text
                            stat[season + ' Usage Percentage'] = player.find('td', {'data-stat':'usg_pct'}).text
                            stat[season + ' Offensive Win Shares'] = player.find('td', {'data-stat':'ows'}).text
                            stat[season + ' Defensive Win Shares'] = player.find('td', {'data-stat':'dws'}).text
                            stat[season + ' Win Shares'] = player.find('td', {'data-stat':'ws'}).text
                            stat[season + ' Win Shares Per 48 Minutes'] = player.find('td', {'data-stat':'ws_per_48'}).text
                            stat[season + ' Offensive Box Plus/Minus'] = player.find('td', {'data-stat':'obpm'}).text
                            stat[season + ' Defensive Box Plus/Minus'] = player.find('td', {'data-stat':'dbpm'}).text
                            stat[season + ' Box Plus/Minus'] = player.find('td', {'data-stat':'bpm'}).text
                            stat[season + ' Value Over Replacement Player'] = player.find('td', {'data-stat':'vorp'}).text 
                            if player.find('td', {'data-stat':'team_id'}).text == 'None':
                                stat[season + ' Team'] = player.find('td', {'data-stat':'team_id'}).find('a').text
                            else:
                                stat[season + ' Team'] = player.find('td', {'data-stat':'team_id'}).text
                            if stat[season + ' Team'] == 'TOT':             
                                stat[season + ' Team'] = player.find('td', {'data-stat':'team_id'}).contents[-1].strip()
                            for i in team_yearly_payrolls: 
                                if season == i[2] and stat[season + ' Team'] == i[1]:
                                    stat[season + ' Total Team Payroll'] = i[0]     
                                else:
                                    continue


                if player_soup.find('div', {'id':'content'}).find('table', {'id':'all_salaries'}) == None:
                    continue
                else:
                    for player in table4.find_all('tr'):
                        season = player.find('th', {'data-stat':'season'}).text
                        stat[season + ' Salary'] = player.find('td', {'data-stat':'salary'}).text
                stats_for_df.append(stat)
            except AttributeError:
                print('Nope')
   
        
        
            
        

2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
2017-18
Nope
2017-18
2017-18
2017-18
2017-18
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2016-17
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2015-16
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2014-15
2013-14
2013-14
2013-14
2013-14
201

In [375]:
stats_for_df[13]

{'2016-17 2 Point Field Goal Attempts Per Game': '1.8',
 '2016-17 2 Point Field Goal Percentage': '.500',
 '2016-17 2 Point Field Goals Per Game': '0.9',
 '2016-17 3 Point Attempt Rate': '.412',
 '2016-17 3 Point Field Goal Attempts Per Game': '1.3',
 '2016-17 3 Point Field Goal Percentage': '.286',
 '2016-17 3 Point Field Goals Per Game': '0.4',
 '2016-17 Age': '21',
 '2016-17 Assist Percentage': '8.7',
 '2016-17 Assists Per Game': '0.5',
 '2016-17 Block Percentage': '0.5',
 '2016-17 Blocks Per Game': '0.0',
 '2016-17 Box Plus/Minus': '-5.5',
 '2016-17 Defensive Box Plus/Minus': '-3.0',
 '2016-17 Defensive Rebound Percentage': '11.5',
 '2016-17 Defensive Rebounds Per Game': '0.9',
 '2016-17 Defensive Win Shares': '0.1',
 '2016-17 Effective Field Goal Percentage': '.471',
 '2016-17 Field Goal Attempts Per Game': '3.1',
 '2016-17 Field Goal Percentage': '.412',
 '2016-17 Field Goals Per Game': '1.3',
 '2016-17 Free Throw Attempt Rate': '.279',
 '2016-17 Free Throw Attempts Per Game': '0

In [376]:
stats = pd.DataFrame(stats_for_df)
stats = stats.drop_duplicates()

In [377]:
stats.shape

(1196, 1662)

In [378]:
stats.to_csv('safeguard.csv')

In [379]:
stats.head()

,1984-85 2 Point Field Goal Attempts Per Game,1984-85 2 Point Field Goal Percentage,1984-85 2 Point Field Goals Per Game,1984-85 3 Point Attempt Rate,1984-85 3 Point Field Goal Attempts Per Game,1984-85 3 Point Field Goal Percentage,1984-85 3 Point Field Goals Per Game,1984-85 Age,1984-85 Assist Percentage,1984-85 Assists Per Game,1984-85 Block Percentage,1984-85 Blocks Per Game,1984-85 Box Plus/Minus,1984-85 Defensive Box Plus/Minus,1984-85 Defensive Rebound Percentage,1984-85 Defensive Rebounds Per Game,1984-85 Defensive Win Shares,1984-85 Effective Field Goal Percentage,1984-85 Field Goal Attempts Per Game,1984-85 Field Goal Percentage,1984-85 Field Goals Per Game,1984-85 Free Throw Attempt Rate,1984-85 Free Throw Attempts Per Game,1984-85 Free Throw Makes Per Game,1984-85 Free Throw Percentage,1984-85 Games Played,1984-85 Games Started,1984-85 Minutes Per Game,1984-85 Offensive Box Plus/Minus,1984-85 Offensive Rebound Percentage,1984-85 Offensive Rebounds Per Game,1984-85 Offensive Win Shares,1984-85 Personal Fouls Per Game,1984-85 Player Efficiency Rating,1984-85 Points Per Game,1984-85 Steal Percentage,1984-85 Steals Per Game,1984-85 Team,1984-85 Total Minutes Played,1984-85 Total Rebound Percentage,1984-85 Total Rebounds Per Game,1984-85 True Shooting Percentage,1984-85 Turnover Percentage,1984-85 Turnovers Per Game,1984-85 Usage Percentage,1984-85 Value Over Replacement Player,1984-85 Win Shares,1984-85 Win Shares Per 48 Minutes,1984-85 Years In League,1985-86 2 Point Field Goal Attempts Per Game,1985-86 2 Point Field Goal Percentage,1985-86 2 Point Field Goals Per Game,1985-86 3 Point Attempt Rate,1985-86 3 Point Field Goal Attempts Per Game,1985-86 3 Point Field Goal Percentage,1985-86 3 Point Field Goals Per Game,1985-86 Age,1985-86 Assist Percentage,1985-86 Assists Per Game,1985-86 Block Percentage,1985-86 Blocks Per Game,1985-86 Box Plus/Minus,1985-86 Defensive Box Plus/Minus,1985-86 Defensive Rebound Percentage,1985-86 Defensive Rebounds Per Game,1985-86 Defensive Win Shares,1985-86 Effective Field Goal Percentage,1985-86 Field Goal Attempts Per Game,1985-86 Field Goal Percentage,1985-86 Field Goals Per Game,1985-86 Free Throw Attempt Rate,1985-86 Free Throw Attempts Per Game,1985-86 Free Throw Makes Per Game,1985-86 Free Throw Percentage,1985-86 Games Played,1985-86 Games Started,1985-86 Minutes Per Game,1985-86 Offensive Box Plus/Minus,1985-86 Offensive Rebound Percentage,1985-86 Offensive Rebounds Per Game,1985-86 Offensive Win Shares,1985-86 Personal Fouls Per Game,1985-86 Player Efficiency Rating,1985-86 Points Per Game,1985-86 Salary,1985-86 Steal Percentage,1985-86 Steals Per Game,1985-86 Team,1985-86 Total Minutes Played,1985-86 Total Rebound Percentage,1985-86 Total Rebounds Per Game,1985-86 True Shooting Percentage,1985-86 Turnover Percentage,1985-86 Turnovers Per Game,1985-86 Usage Percentage,1985-86 Value Over Replacement Player,1985-86 Win Shares,1985-86 Win Shares Per 48 Minutes,1985-86 Years In League,1986-87 2 Point Field Goal Attempts Per Game,1986-87 2 Point Field Goal Percentage,1986-87 2 Point Field Goals Per Game,1986-87 3 Point Attempt Rate,1986-87 3 Point Field Goal Attempts Per Game,1986-87 3 Point Field Goal Percentage,1986-87 3 Point Field Goals Per Game,1986-87 Age,1986-87 Assist Percentage,1986-87 Assists Per Game,1986-87 Block Percentage,1986-87 Blocks Per Game,1986-87 Box Plus/Minus,1986-87 Defensive Box Plus/Minus,1986-87 Defensive Rebound Percentage,1986-87 Defensive Rebounds Per Game,1986-87 Defensive Win Shares,1986-87 Effective Field Goal Percentage,1986-87 Field Goal Attempts Per Game,1986-87 Field Goal Percentage,1986-87 Field Goals Per Game,1986-87 Free Throw Attempt Rate,1986-87 Free Throw Attempts Per Game,1986-87 Free Throw Makes Per Game,1986-87 Free Throw Percentage,1986-87 Games Played,1986-87 Games Started,1986-87 Minutes Per Game,1986-87 Offensive Box Plus/Minus,1986-87 Offensive Rebound Percentage,1986-87 Offensive Rebounds Per Game,1986-87 Offensive Win Shares,1986-

In [385]:
stats['2000-01 Salary'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: 2000-01 Salary, dtype: object

In [386]:
def floatify(col):
    for i in range(0,len(col)):
        try:
            if col[i] == None:
                continue
            if col.isnull()[i] == True:
                continue
            else:
                col[i] = col[i].replace(',','')
                col[i] = col[i].replace('$','')
        except KeyError:
            continue

In [387]:
to_float = [stats['2003-04 Salary'], stats['2004-05 Salary'], stats['2005-06 Salary'], stats['2006-07 Salary'],
           stats['2007-08 Salary'], stats['2008-09 Salary'], stats['2009-10 Salary'], stats['2010-11 Salary'],
           stats['2011-12 Salary'], stats['2012-13 Salary'], stats['2013-14 Salary'], stats['2014-15 Salary'],
           stats['2015-16 Salary'], stats['2016-17 Salary']]

In [388]:
for i in to_float:
    floatify(i)
    print("Floated")

/Users/jordanarnold22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/jordanarnold22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Floated
Floated
Floated
Floated
Floated
Floated
Floated
Floated
Floated
Floated
Floated
Floated
Floated
Floated


In [389]:
stats = stats.convert_objects(convert_numeric=True)

/Users/jordanarnold22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [390]:
contracts = pd.read_csv('OTHER/contracts.csv')

In [391]:
contracts.drop('Name.1', axis=1, inplace=True)
contracts.drop('Team', axis=1, inplace=True)

In [392]:
contracts

,Name,2017-18 Salary,2018-19 Salary,2019-20 Salary,2020-21 Salary,Contract Type
0,Stephen Curry,34682550,37457154.0,40231758.0,43006362.0,Bird Rights
1,LeBron James,33285709,NaN,NaN,NaN,Early Bird
2,Paul Millsap,31269231,29730769.0,30500000.0,NaN,Cap space
3,Gordon Hayward,29727900,31214295.0,32700690.0,34187085.0,Cap space
4,Blake Griffin,29512900,31873932.0,34234964.0,36595996.0,Bird Rights
5,Kyle Lowry,28703704,31000000.0,33296296.0,NaN,Bird Rights
6,Russell Westbrook,28530608,35350000.0,38178000.0,41006000.0,Bird Rights
7,Mike Conley,28530608,30521115.0,32511623.0,34504132.0,Cap Space
8,James Harden,28299399,30431854.0,37800000.0,40824000.0,Bird Rights
9,DeMar DeRozan,27739975,27739975.0,27739975.0,27739975.0,Cap Space


In [393]:
stats.shape

(1196, 1662)

In [394]:
final = pd.merge(stats, contracts, on='Name', how='outer')

In [395]:
final = final.drop_duplicates()

In [396]:
final.shape

(1324, 1667)

In [397]:
final.head()

,1984-85 2 Point Field Goal Attempts Per Game,1984-85 2 Point Field Goal Percentage,1984-85 2 Point Field Goals Per Game,1984-85 3 Point Attempt Rate,1984-85 3 Point Field Goal Attempts Per Game,1984-85 3 Point Field Goal Percentage,1984-85 3 Point Field Goals Per Game,1984-85 Age,1984-85 Assist Percentage,1984-85 Assists Per Game,1984-85 Block Percentage,1984-85 Blocks Per Game,1984-85 Box Plus/Minus,1984-85 Defensive Box Plus/Minus,1984-85 Defensive Rebound Percentage,1984-85 Defensive Rebounds Per Game,1984-85 Defensive Win Shares,1984-85 Effective Field Goal Percentage,1984-85 Field Goal Attempts Per Game,1984-85 Field Goal Percentage,1984-85 Field Goals Per Game,1984-85 Free Throw Attempt Rate,1984-85 Free Throw Attempts Per Game,1984-85 Free Throw Makes Per Game,1984-85 Free Throw Percentage,1984-85 Games Played,1984-85 Games Started,1984-85 Minutes Per Game,1984-85 Offensive Box Plus/Minus,1984-85 Offensive Rebound Percentage,1984-85 Offensive Rebounds Per Game,1984-85 Offensive Win Shares,1984-85 Personal Fouls Per Game,1984-85 Player Efficiency Rating,1984-85 Points Per Game,1984-85 Steal Percentage,1984-85 Steals Per Game,1984-85 Team,1984-85 Total Minutes Played,1984-85 Total Rebound Percentage,1984-85 Total Rebounds Per Game,1984-85 True Shooting Percentage,1984-85 Turnover Percentage,1984-85 Turnovers Per Game,1984-85 Usage Percentage,1984-85 Value Over Replacement Player,1984-85 Win Shares,1984-85 Win Shares Per 48 Minutes,1984-85 Years In League,1985-86 2 Point Field Goal Attempts Per Game,1985-86 2 Point Field Goal Percentage,1985-86 2 Point Field Goals Per Game,1985-86 3 Point Attempt Rate,1985-86 3 Point Field Goal Attempts Per Game,1985-86 3 Point Field Goal Percentage,1985-86 3 Point Field Goals Per Game,1985-86 Age,1985-86 Assist Percentage,1985-86 Assists Per Game,1985-86 Block Percentage,1985-86 Blocks Per Game,1985-86 Box Plus/Minus,1985-86 Defensive Box Plus/Minus,1985-86 Defensive Rebound Percentage,1985-86 Defensive Rebounds Per Game,1985-86 Defensive Win Shares,1985-86 Effective Field Goal Percentage,1985-86 Field Goal Attempts Per Game,1985-86 Field Goal Percentage,1985-86 Field Goals Per Game,1985-86 Free Throw Attempt Rate,1985-86 Free Throw Attempts Per Game,1985-86 Free Throw Makes Per Game,1985-86 Free Throw Percentage,1985-86 Games Played,1985-86 Games Started,1985-86 Minutes Per Game,1985-86 Offensive Box Plus/Minus,1985-86 Offensive Rebound Percentage,1985-86 Offensive Rebounds Per Game,1985-86 Offensive Win Shares,1985-86 Personal Fouls Per Game,1985-86 Player Efficiency Rating,1985-86 Points Per Game,1985-86 Salary,1985-86 Steal Percentage,1985-86 Steals Per Game,1985-86 Team,1985-86 Total Minutes Played,1985-86 Total Rebound Percentage,1985-86 Total Rebounds Per Game,1985-86 True Shooting Percentage,1985-86 Turnover Percentage,1985-86 Turnovers Per Game,1985-86 Usage Percentage,1985-86 Value Over Replacement Player,1985-86 Win Shares,1985-86 Win Shares Per 48 Minutes,1985-86 Years In League,1986-87 2 Point Field Goal Attempts Per Game,1986-87 2 Point Field Goal Percentage,1986-87 2 Point Field Goals Per Game,1986-87 3 Point Attempt Rate,1986-87 3 Point Field Goal Attempts Per Game,1986-87 3 Point Field Goal Percentage,1986-87 3 Point Field Goals Per Game,1986-87 Age,1986-87 Assist Percentage,1986-87 Assists Per Game,1986-87 Block Percentage,1986-87 Blocks Per Game,1986-87 Box Plus/Minus,1986-87 Defensive Box Plus/Minus,1986-87 Defensive Rebound Percentage,1986-87 Defensive Rebounds Per Game,1986-87 Defensive Win Shares,1986-87 Effective Field Goal Percentage,1986-87 Field Goal Attempts Per Game,1986-87 Field Goal Percentage,1986-87 Field Goals Per Game,1986-87 Free Throw Attempt Rate,1986-87 Free Throw Attempts Per Game,1986-87 Free Throw Makes Per Game,1986-87 Free Throw Percentage,1986-87 Games Played,1986-87 Games Started,1986-87 Minutes Per Game,1986-87 Offensive Box Plus/Minus,1986-87 Offensive Rebound Percentage,1986-87 Offensive Rebounds Per Game,1986-87 Offensive Win Shares,1986-

In [398]:
final.to_csv('all_historical_stats.csv')